### Coleta usando a API do twitter

In [1]:
import oauth2
import sys
import os
import json
import time
import csv
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
import threading
import re

In [2]:
def criaJsonSaida(json_input):
    return {
        "id": json_input["id"],
        "data_criacao": json_input["created_at"],
        "text": json_input["full_text"].lower(),
        "usuario": json_input["user"]["name"],
        "e_sobre_a_empresa": None
    }

def get_requisicao(cliente, max_id, termo):
    requisicao = cliente.request(
        'https://api.twitter.com/1.1/search/tweets.json?q={termo}&tweet_mode=extended&result_type=recent&lang=pt-br&count=3&{max_id}'.format(max_id=max_id, termo=termo))
    decodificar = requisicao[1].decode()
    try:
        objeto = json.loads(decodificar)
        return objeto['statuses']
    except Exception:
        print(objeto)

def coleta(api_key, api_secret_key, token_key, token_secret_key, termo, num_tweets, arquivo_saida):
    consumer = oauth2.Consumer(api_key, api_secret_key)
    token = oauth2.Token(token_key, token_secret_key)
    cliente = oauth2.Client(consumer, token)
    strUrl = ""
    max_id = ""
    num_tweets_collected = 0

    tweetsColetados = {
        "tweets": []
    }

    dataset = {}
    
    try:
        # coleta os tweets
        while(num_tweets_collected < num_tweets):
            tweets = get_requisicao(cliente, max_id, termo)            
            for tweet in tweets:
                tweet_saida = criaJsonSaida(tweet)
                par_unico = (tweet_saida["usuario"], tweet_saida["data_criacao"])
                if dataset.get(par_unico) == None:
                    dataset[par_unico] = { "id" : tweet_saida["id"]}
                    tweetsColetados["tweets"] += [criaJsonSaida(tweet)]
                
                max_id = "max_id={max_id}&".format(max_id=tweet['id'])
                num_tweets_collected += 1
    except Exception as e:
        print(e)
    finally:
        with open(arquivo_saida, 'w') as f:
            f.write(json.dumps(tweetsColetados["tweets"], indent=2))


In [4]:
# inicializa keys
api_key = 'suaAPIKey'
api_secret_key = 'suaAPISecretKey'
token_key = 'suaTokenKey'
token_secret_key = 'suaTokenSecretKey'
termo = 'sadia'
num_tweets = 300
arquivo_com_tweetes_coletados = 'tweets_coleta_iterativa.json'

try:
    coleta(api_key, api_secret_key, token_key,
           token_secret_key, termo, num_tweets, arquivo_com_tweetes_coletados)
except Exception as e:
    print(e)

{'errors': [{'code': 88, 'message': 'Rate limit exceeded'}]}
'NoneType' object is not iterable


### Gerando csv com os tweets a serem rotulados

In [12]:
with open(arquivo_com_tweetes_coletados, 'r') as f:
    dic_tweets = json.loads(f.read())
    tweetsDF = pd.read_json(json.dumps(dic_tweets))
nome_arquivo_com_tweetes_rotulados = 'tweets.csv'
tweetsDF.to_csv(nome_arquivo_com_tweetes_rotulados, quoting=csv.QUOTE_ALL)

### Lendo o csv com os tweets rotulados

In [13]:
tweets_rotuladosDF = pd.read_csv(nome_arquivo_com_tweetes_rotulados)

In [14]:
tweets_rotuladosDF

,Unnamed: 0,data_criacao,e_sobre_a_empresa,id,text,usuario
0,0,Fri Apr 19 21:30:03 +0000 2019,NaN,1.119352e+18,@fatmaiconkkk @MAlCONKUSTERKKK @cacaushow @Bur...,Canto da sereia kuster
1,1,Fri Apr 19 21:02:33 +0000 2019,NaN,1.119346e+18,Esse rage e a necessidade de expor o q tô pens...,Cauê Petito
2,2,Fri Apr 19 20:53:17 +0000 2019,NaN,1.119343e+18,"Que mané peixe, almoço de hoje foi Lasanha Sad...",Suricate
3,3,Fri Apr 19 20:47:39 +0000 2019,NaN,1.119342e+18,Esses Strogonoff da Sadia são bom pra caralho,desesperado porém tranquilo
4,4,Thu Apr 18 22:48:41 +0000 2019,NaN,NaN,@leo_godinho 99 seguidores foi uma brincadeiri...,FelippeCardosoN
5,5,Thu Apr 18 22:33:54 +0000 2019,NaN,NaN,Estamos muito longe de uma sociedade sadia... ...,vinilabanca
6,6,Thu Apr 18 22:13:08 +0000 2019,NaN,NaN,Gnt eu sugeri para o boy ele fazer pizza pra m...,_anneway
7,7,Thu Apr 18 22:05:59 +0000 2019,NaN,NaN,"Perdigão ou Sadia? — Me diz você, prefere qual...",Salles7crf
8,8,Thu Apr 18 21:52:10 +0000 2019,NaN,NaN,Primeira vez na vida q eu e a testuda tivemos ...,carolz1nh
9,9,Thu Apr 18 22:45:30 +0000 2019,NaN,NaN,Esse Ciro atualizado de mente aberta e sadia a...,Edilbertolorde


### Fazendo um pré-processamento nos textos dos tweets

### Identificando os brigramas e trigramas encontrados nos twittes

In [15]:
tknzr = TweetTokenizer()

ngram = {}

for indice, linha in tweets_rotuladosDF.iterrows() :
    tokens = tknzr.tokenize(linha[tweets_rotuladosDF.text.name])
    for count in range(len(tokens)):
        if(tokens[count] == 'Sadia' or tokens[count] == 'sadia'):
            bigram = (tokens[count - 1], tokens[count])
            trigram = (tokens[count - 2], tokens[count - 1], tokens[count])
            if ngram.get(bigram) == None:
                ngram[bigram] = {"frequencia" : 1, "id_tweets" : [indice]}
            else:
                valor = ngram.get(bigram)
                valor["frequencia"] += 1
                valor["id_tweets"] += [indice]
                
            if ngram.get(trigram) == None:
                ngram[trigram] = {"frequencia" : 1, "id_tweets" : [indice]}
            else:
                valor = ngram.get(trigram)
                valor["frequencia"] += 1
                valor["id_tweets"] += [indice]
                

In [16]:
ngram = sorted(ngram.items(), key=lambda x: x[1]['frequencia'], reverse=True)

Imprimindo os n mais frequetes

In [17]:
n = 100
contador = 1
for tupla in ngram:
    if(contador > n):
        break
    chaves = tupla[0]
    valores = tupla[1]
    print("freq: " + str(valores['frequencia']) + " " + str(chaves))
    contador += 1

freq: 5 ('da', 'Sadia')
freq: 4 ('sociedade', 'sadia')
freq: 4 ('Perdigão', 'ou', 'Sadia')
freq: 4 ('uma', 'sociedade', 'sadia')
freq: 4 ('ou', 'Sadia')
freq: 4 ('uma', 'da', 'Sadia')
freq: 2 ('&', 'sadia')
freq: 2 ('brincadeirinha', 'sadia')
freq: 2 ('uma', 'conversa', 'sadia')
freq: 2 ('uma', 'brincadeirinha', 'sadia')
freq: 2 ('jovem', '&', 'sadia')
freq: 2 ('aberta', 'e', 'sadia')
freq: 2 ('conversa', 'sadia')
freq: 2 ('e', 'sadia')
freq: 1 ('e', 'gente', 'sadia')
freq: 1 ('gente', 'sadia')
freq: 1 ('foi', 'Lasanha', 'Sadia')
freq: 1 ('Strogonoff', 'da', 'Sadia')
freq: 1 ('Lasanha', 'Sadia')
